In [1]:
import copy
import numpy as np
import numpy.linalg as la
import time
from collections import deque
from operator import itemgetter
import sys
import queue as Q
from itertools import permutations


In [2]:
class graph():
    
    def __init__(self, dict_path_dist):
    #         self.start = dict_path_dist[0]\n",
#         self.start = 
        self.nodes = []
        self.edges = {}
        self.adjancent = {}
        for node in dict_path_dist:
            self.insert_node(node)
            for x in self.nodes:
                for y in self.nodes:
                    if x != y:
                        self.insert_edge(x, y, dict_path_dist[x][1][y])
    def insert_node(self, node):
        self.nodes.append(node)

    def insert_edge(self, x, y, weight):
        if (x, y) not in self.edges:
            self.edges[(x, y)] = weight
        if x not in self.adjancent:
            self.adjancent[x] = [y]
        else:    
            self.adjancent[x].append(y)
        
    def get_edge_by_last(self, edges, y):
        output = []
        for x in self.nodes:
            if (x, y) in edges and x != (4,4):
                output.append(((x, y) ,self.edges[x, y]) )
        return output
    
    def del_edge_by_last(self, edges, y):
        for x in self.nodes:
            if (x, y) in edges:
                del edges[(x, y)]
        return edges

    def del_edge_by_first(self, edges, x):
        for y in self.nodes:
            if(x, y) in edges:
                del edges[(x, y)]
        return edges
        

In [3]:
class maze():
    
#   create a maze with list;
    def __init__(self, maze="mediumMaze.txt"):
        self.cost = {}
        self.raw_list = []
        self.maze_list = []
        self.nodes = []
        self.width = None
        self.height = None
        self.end = []
        self.start = None
        self.visited = [] 
        self.solution = [] # ready for draw;
        self.expandedNumber = 0
        
        with open(maze) as f:
            for line in f:
                self.raw_list.append(line.rstrip())
        self.width = len(self.raw_list[0])
        self.height = len(self.raw_list)
        
        for line in self.raw_list:
            for char in line:
                if char =="%":
                    self.maze_list.append(0)
                elif char ==".":
#                     self.maze_list.append("G")
                    self.maze_list.append(1)
                    self.end.append((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))
                    self.nodes.append((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))
                elif char =="P":
#                     self.maze_list.append("P")
                    self.maze_list.append(1)
                    self.start = ((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))
                    self.nodes.append((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))
                else:
                    self.maze_list.append(1)
                    self.nodes.append((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))

    def accessMaze(self, x, y):
        return self.maze_list[self.getPoint(x, y)]                
    
#   Transform the index in the raw_list to the index in the maze_list
    def getPoint(self, x, y):
        if x < self.width and y < self.height:
            return y*self.width + x
        else:
            print("Index out of bound!")
            return False
        
    def getCordinate(self, point):
        return (point%self.width, point//self.width)
    
    
#   Return the adjacant node who is unvisited and valid
    def adjacentList(self, pos):
        adjacent = []
        x, y = pos
        if self.accessMaze(x+1, y) and self.getPoint(x+1, y) not in self.visited:
            adjacent.append((x+1, y))
        if self.accessMaze(x, y+1) and self.getPoint(x, y+1) not in self.visited:
            adjacent.append((x, y+1))
        if self.accessMaze(x-1, y) and self.getPoint(x-1, y) not in self.visited:
            adjacent.append((x-1, y))
        if self.accessMaze(x, y-1) and self.getPoint(x, y-1) not in self.visited:
            adjacent.append((x, y-1))
        return adjacent


    def manhattanDistance(self, cur, des): return abs(cur[0] - des[0]) + abs(cur[1] - des[1])

    def collect_all_mst(self):
        all_nodes = list(self.end)
        all_nodes.append(self.start)
        dict_path_dist = {}
        for n in all_nodes:
            shortestPathTree, disToStart = self.dijsktra(n)
            dict_path_dist[n] = [shortestPathTree, disToStart]
        return dict_path_dist
    
    
    def create_graph(self):
        self.g = graph(self.collect_all_mst())
        return self.g
    
    
       
    def collect_all_mst(self):
        all_nodes = list(self.end)
        all_nodes.append(self.start)
        dict_path_dist = {}
        for n in all_nodes:
            shortestPathTree, disToStart = self.dijsktra(n)
            dict_path_dist[n] = [shortestPathTree, disToStart]
        return dict_path_dist
    
    def create_graph(self):
        self.g = graph(self.collect_all_mst())
        return self.g
      
          
      
    def dynammic(self):
        self.g = self.create_graph()
        nodes = self.g.nodes[:]
        nodes.remove(self.start)
        self.edges = copy.deepcopy(self.g.edges)
        
        self.memo = {}
        cost = []
          
        for n in nodes:
            temp = nodes[:]
            temp.remove(n)
#             print(self.memo)
            if (tuple(temp), n) in self.memo:
                cost.append(self.memo[(tuple(temp), n)])
            else:
                self.memo[(tuple(temp), n)] = self.dynammic_helper(temp, n)
                cost.append(self.memo[(tuple(temp), n)])
#             print(n)
#         print(cost)
        return min(cost)
              
    def dynammic_helper(self, nodes, y):
        if not nodes:
#             print(\"touch the start\")
            if (self.start, y) not in self.memo:
                t = self.edges[(self.start, y)]
                self.memo[(self.start, y)] = t
            return t
        new_cost = []
        for n in nodes:
#             with Pool(5) as p:
                
            temp = nodes[:]
            temp.remove(n)
            if (tuple(temp), n) not in self.memo:
                t = self.dynammic_helper(temp, n)
                self.memo[(tuple(temp), n)] = t
                new_cost.append(t + self.edges[(n, y)])
            else:
                new_cost.append(self.memo[(tuple(temp), n)] + self.edges[(n ,y)])
    #             print(n, y, self.g.edges[(n, y)])
#             print(new_cost)
        return min(new_cost)
      

      
    
    
    
    
    
    

#   Create MST; Find the shortest path;
    def dijsktra(self, start):
#       Record the path; key: node   value: pre;
        shortestPathTree = {start: (-1,-1)}
#       Distance from the node to the starting point; key: node   value: distance;
        disToStart = {start:0}
        
#       Initialize the distance record; ESP, starting point has distance 0 and other nodes has maxINT;
        for node in range(self.width*self.height):
            if self.getCordinate(node) != start and self.maze_list[node] != 0:
                disToStart[self.getCordinate(node)] = sys.maxsize
#       self.nodes saves all the nodes; Check __init__;
        nodes = self.nodes[:]
#       Travel all the nodes, label their distance and pre;
        while nodes:
            min_node = None
#           Find the visited node with least distance;
            for node in nodes:
                if node in shortestPathTree:
                    if min_node == None:
                        min_node = node
                    elif disToStart[node] < disToStart[min_node]:
                        min_node = node
#           All the nodes have already been checked;
            if min_node == None:
                break
            
            nodes.remove(min_node)
#           Current cost from start to the node;
            minToStart = disToStart[min_node]
            
            adjacent_nodes = self.adjacentList(min_node)
#           Label all the adjacent nodes;
            for adj_node in adjacent_nodes:
#               The edge weight is only 1;
                newToStart = minToStart + 1
#                 newToStart = minToStart + self.manhattanDistance(min_node, self.end)
#               If the node has not been visited or the distance should be updated;
                if adj_node not in shortestPathTree or newToStart < disToStart[adj_node]:
                    shortestPathTree[adj_node] = min_node
                    disToStart[adj_node] = newToStart
        return shortestPathTree, disToStart

In [4]:
# m = maze("mediumSearch.txt")
m = maze("smallSearch.txt")
# m.create_graph()
# print(len(m.g.edges))
# print(len(m.g.nodes))
# m.g.adjancent[1,1]
# m.calculate_cost()
m.dynammic()
# m.g.nodes

124

In [7]:
m.g.adjancent

{(1, 1): [(8, 1),
  (3, 2),
  (6, 3),
  (8, 3),
  (2, 4),
  (1, 5),
  (4, 5),
  (7, 5),
  (8, 6),
  (1, 7),
  (6, 7),
  (4, 4)],
 (1, 5): [(1, 1),
  (8, 1),
  (3, 2),
  (6, 3),
  (8, 3),
  (2, 4),
  (4, 5),
  (7, 5),
  (8, 6),
  (1, 7),
  (6, 7),
  (4, 4)],
 (1, 7): [(1, 1),
  (8, 1),
  (3, 2),
  (6, 3),
  (8, 3),
  (2, 4),
  (1, 5),
  (4, 5),
  (7, 5),
  (8, 6),
  (6, 7),
  (4, 4)],
 (2, 4): [(1, 1),
  (8, 1),
  (3, 2),
  (6, 3),
  (8, 3),
  (1, 5),
  (4, 5),
  (7, 5),
  (8, 6),
  (1, 7),
  (6, 7),
  (4, 4)],
 (3, 2): [(1, 1),
  (8, 1),
  (6, 3),
  (8, 3),
  (2, 4),
  (1, 5),
  (4, 5),
  (7, 5),
  (8, 6),
  (1, 7),
  (6, 7),
  (4, 4)],
 (4, 4): [(1, 1),
  (8, 1),
  (3, 2),
  (6, 3),
  (8, 3),
  (2, 4),
  (1, 5),
  (4, 5),
  (7, 5),
  (8, 6),
  (1, 7),
  (6, 7)],
 (4, 5): [(1, 1),
  (8, 1),
  (3, 2),
  (6, 3),
  (8, 3),
  (2, 4),
  (1, 5),
  (7, 5),
  (8, 6),
  (1, 7),
  (6, 7),
  (4, 4)],
 (6, 3): [(1, 1),
  (8, 1),
  (3, 2),
  (8, 3),
  (2, 4),
  (1, 5),
  (4, 5),
  (7, 5),
  (8, 6),

In [36]:
m.start

(25, 8)

In [16]:
from multiprocessing import Pool
def f(x):
    if x > 1:
        return f(x-1)
    return x
with Pool(5) as p:
    print(p.map(f, [1, 2, 3]))

[1, 1, 1]


In [19]:
list(map(lambda x: x, [1, 2, 3]))

[1, 2, 3]

In [14]:
test = {}
if ((1),1)  in test:
    print("jj")

In [10]:
a = [1,2,3]

In [11]:
tuple(a)

(1, 2, 3)

In [12]:
a

[1, 2, 3]